<img src='https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fscoreboredsports.com%2Fwp-content%2Fuploads%2F2016%2F06%2FClippers-logo.png&f=1&nofb=1' style='height: 128px; float:right'/>

# LA Clippers Lineup Analysis

Author: Alex Nakagawa

Last Updated: May 18, 2020

<div class='alert alert-info'>
<b>ALERT</b>: Please ensure that you have switched to a kernel that contains the environment variables listed below.
</div>

In [2]:
import json
import glob
import re
import os
import sys

import psycopg2
import sqlalchemy
import pandas as pd
import numpy as np

assert psycopg2.__version__ == '2.8.5 (dt dec pq3 ext lo64)'
assert sqlalchemy.__version__ == '1.3.16'
assert pd.__version__ == '1.0.3'
assert np.__version__ == '1.18.2'

In [3]:
# Environment variables
DATABASE_HOST = os.environ.get('DATABASE_HOST')
POSTGRES_USER = os.environ.get('POSTGRES_USER')
POSTGRES_PASSWORD = os.environ.get('POSTGRES_PASSWORD') 
DATABASE_NAME = os.environ.get('DATABASE_NAME')

# Ensure that all environment variables are not a None type
assert DATABASE_HOST and POSTGRES_USER and POSTGRES_PASSWORD and DATABASE_NAME

In [4]:
assert sys.version_info[0] == 3

In [5]:
!python3 --version

Python 3.8.1


## 1. Table Creation/Update in PostgreSQL

a. Write code to transfer the files from a directory called `dev_test_data` to a SQL database called `lac_dev_lineups` (code can be Python, SQL, etc.)

i. The tables created should be named `team`, `player`, `game_schedule`, and `lineup`

ii. Make sure your code creates tables if needed and that it can handle data reloads, merges, and/or
updates


### Determining Schema

The following code creates a python dictionary containing the contents of the jsons given by extracting from a `directory`, which in this case is `dev_test_data`.

In [6]:
# Determining the schema needed to design the database

def create_data_map(directory):
    data_file_paths = glob.glob("{}/*.json".format(directory))
    data = {} # Dictionary of python-encoded json objects
    for f in data_file_paths:
        search_obj = re.search(r'{}/(.+).json'.format(directory), f, re.M|re.I) 
        subfile_name = search_obj.group(1) # Return file name
        with open(f, "r") as read_file:
            data[subfile_name] = json.load(read_file)
    return data

In [7]:
data = create_data_map('dev_test_data')
assert data # Make sure data is not empty

From a quick glance at the json load given to us, I've established several keys and important structural notes:

<img src='./images/lineups_schema.jpg' style='height: 400px' />

The next block establishes the connection with the database. The database I've configured is a Cloud SQL Instance in the Google Cloud Platform, which is hosting our PostgreSQL 12 database named `lac_dev_lineups`. Configurations are listed as follows:

* GCP Project ID: **`clippers-test-1`**
* CloudSQL Instance ID: **`clippers-test-instance`**
* PostgreSQL Version: **12**
* \# vCPUs: **1**
* Memory: **3.75GB**
* SSD Storage: **10GB**

A snapshot of the current running GCP project (`clippers-test-1`) is attached.

<img src='./images/gcp_console.jpg' style='height: 256px' />



<div class="alert alert-info">
    You <b>must</b> run the following block to connect to the SQLAlchemy engine. If the connection fails, there may be a need to check the validity of the enviornment variables.

</div>

In [8]:
from sqlalchemy import create_engine

# PostgreSQL + SQLAlchemy.
ENGINE_STRING = 'postgresql+psycopg2://{}:{}@{}/{}'.format(POSTGRES_USER,
                                                           POSTGRES_PASSWORD,
                                                           DATABASE_HOST,
                                                           DATABASE_NAME)

try:
    engine = sqlalchemy.create_engine(ENGINE_STRING, echo=False)
    print("Connection SUCCESS.")
except:
    print("Connection to database failed. Check environment variables.")


Connection SUCCESS.


This is an important function to check whether the tables' `names` exists inside of our `lac_dev_lineups` database.

In [9]:
# Function adapted from https://stackoverflow.com/questions/40652938/flask-sqlalchemy-check-if-table-exists-in-database
def tables_exist(names):
    all_exist = True
    for name in names:
        ret = engine.dialect.has_table(engine, name)
        print('Table "{}" exists: {}'.format(name, ret))
        all_exist = all_exist and ret
    return all_exist

### Creating Tables if they do not exist

The following code takes advantage of what is known in SQLAlchemy as **[Object Relational Mapping (ORM)](https://www.tutorialspoint.com/sqlalchemy/sqlalchemy_orm_declaring_mapping.htm)**. SQLAlchemy takes care of most of the serialization involved between Python's class/object definitions and PostgreSQL's type definitions.

In [10]:
# Declarative mapping of Objects if necessary
from sqlalchemy import Column, ForeignKey, Numeric, Integer, String, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker

Base = declarative_base()

class Team(Base):
    ''' * = primary key
        ----------------------
       |team                  |
       |----------------------|
       |col       |type       |
       |----------------------|
       |team_id*  |INTEGER    |
       |name      |STRING(250)|
       |city      |STRING(250)|
       |abrv      |STRING(250)|
        ----------------------
    '''
    __tablename__ = 'team'
    team_id = Column(Integer, primary_key=True, nullable=False) # primary key
    name = Column(String(length=250), nullable=False)
    city = Column(String(length=250), nullable=False)
    abrv = Column(String(length=250), nullable=False)
    
    
class Player(Base):
    ''' * = primary key
        ----------------------
       |player                |
       |----------------------|
       |col       |type       |
       |----------------------|
       |player_id*|INTEGER    |
       |first_name|STRING(250)|
       |last_name |STRING(250)|
        ----------------------
    '''
    __tablename__ = 'player'
    player_id = Column(Integer, primary_key=True, nullable=False) # primary key
    first_name = Column(String(length=250), nullable=False)
    last_name = Column(String(length=250), nullable=False)

    
class Game(Base):
    ''' *  = primary key
        ** = foreign key
        ----------------------
       |game_schedule         |
       |----------------------|
       |col       |type       |
       |----------------------|
       |game_id*  |INTEGER    |
       |home_id** |INTEGER    |
       |home_score|INTEGER    |
       |away_id** |INTEGER    |
       |away_score|INTEGER    |
       |game_date |TIMESTAMP  |
        ----------------------
    '''
    __tablename__ = 'game_schedule'
    game_id = Column(Integer, primary_key=True, nullable=False) # primary key
    home_id = Column(Integer, ForeignKey('team.team_id'), nullable=False)
    home_score = Column(Integer) # nullable in case game hasn't happened yet
    away_id = Column(Integer, ForeignKey('team.team_id'), nullable=False)
    away_score = Column(Integer) # nullable in case game hasn't happened yet
    game_date = Column(DateTime(timezone=False))
    
    

class Lineup(Base):
    ''' *  = primary keys
        ** = foreign key
        ------------------------
       |lineup                  |
       |------------------------|
       |col         |type       |
       |------------------------|
       |team_id***  |INTEGER    |
       |player_id***|INTEGER    |
       |game_id***  |INTEGER    |
       |lineup_num* |INTEGER    |
       |period*     |INTEGER    |
       |time_in     |DECIMAL    |
       |time_out    |DECIMAL    |
        ------------------------
    '''
    __tablename__ = 'lineup'
    team_id = Column(Integer, ForeignKey("team.team_id"), primary_key=True)
    player_id = Column(Integer, ForeignKey("player.player_id"), primary_key=True)
    game_id = Column(Integer, ForeignKey("game_schedule.game_id"), primary_key=True)
    lineup_num = Column(Integer, primary_key=True)
    period = Column(Integer, primary_key=True, nullable=False)
    time_in = Column(Numeric, nullable=False)
    time_out = Column(Numeric, nullable=False)
    
    
def create_tables(engine):
    Session = sessionmaker(bind = engine)
    session = Session()
    try:
        table_names = ['team', 'player', 'game_schedule', 'lineup']
        if not tables_exist(table_names):
            Base.metadata.create_all(engine)
        else:
            print("Table names {} all exist. No new tables were created.".format(table_names))
    except:
        session.rollback()
        raise
    finally:
        session.close()

In [11]:
create_tables(engine)

Table "team" exists: True
Table "player" exists: True
Table "game_schedule" exists: True
Table "lineup" exists: True
Table names ['team', 'player', 'game_schedule', 'lineup'] all exist. No new tables were created.


In [12]:
from datetime import datetime
from sqlalchemy import exc

def create_teams_list(team_json):
    '''Create a list of Pythonic Team objects, prepare for insertion to table team'''
    team_list = []
    for j in team_json:
        team = Team(team_id = j["team_id"],
                 name = j["name"],
                 city = j["city"],
                 abrv = j["abrv"])
        team_list.append(team)
    return team_list

def create_players_list(player_json):
    '''Create a list of Pythonic Player objects, prepare for insertion to table player'''
    player_list = []
    for j in player_json:
        player = Player(player_id=j['player_id'],
                        first_name=j['first_name'],
                        last_name=j['last_name'])
        player_list.append(player)
    return player_list

def create_games_list(game_json):
    '''Create a list of Pythonic Game objects, prepare for insertion to table game_schedule'''
    game_list = []
    for j in game_json:
        game = Game(game_id=j['game_id'],
                    home_id=j['home_id'],
                    home_score=j['home_score'],
                    away_id=j['away_id'],
                    away_score=j['away_score'],
                    game_date=datetime.strptime(j['game_date'], "%Y-%m-%d %H:%M:%S"))
        game_list.append(game)
    return game_list

def create_lineups_list(lineup_json):
    '''Create a list of Pythonic Lineup objects, prepare for insertion to table lineup'''
    lineup_list = []
    for j in lineup_json:
        lineup = Lineup(team_id=j['team_id'],
                    player_id=j['player_id'],
                    game_id=j['game_id'],
                    lineup_num=j['lineup_num'],
                    period=j['period'],
                    time_in=j['time_in'],
                    time_out=j['time_out'])
        lineup_list.append(lineup)
    return lineup_list


def insert_records(record_list):
    '''Takes a record_list and add them to a session, and eventually the database.'''
    Session = sessionmaker(bind = engine)
    session = Session()
    
    try:
        session.add_all(record_list)
        session.commit()
    except exc.IntegrityError as error:
        print("You attempted to add a new value, but there was a duplicate key: {}".format(error))
        session.rollback()
    except:
        session.rollback()
        raise
    finally:
        session.close()
 

In [13]:
insert_records(create_teams_list(data['team']))
insert_records(create_players_list(data['player']))
insert_records(create_games_list(data['game_schedule']))
insert_records(create_lineups_list(data['lineup']))

You attempted to add a new value, but there was a duplicate key: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "team_pkey"
DETAIL:  Key (team_id)=(1) already exists.

[SQL: INSERT INTO team (team_id, name, city, abrv) VALUES (%(team_id)s, %(name)s, %(city)s, %(abrv)s)]
[parameters: ({'team_id': 1, 'name': 'Hawks', 'city': 'Atlanta', 'abrv': 'ATL'}, {'team_id': 2, 'name': 'Celtics', 'city': 'Boston', 'abrv': 'BOS'}, {'team_id': 3, 'name': 'Nets', 'city': 'Brooklyn', 'abrv': 'BKN'}, {'team_id': 4, 'name': 'Hornets', 'city': 'Charlotte', 'abrv': 'CHA'}, {'team_id': 5, 'name': 'Bulls', 'city': 'Chicago', 'abrv': 'CHI'}, {'team_id': 6, 'name': 'Cavaliers', 'city': 'Cleveland', 'abrv': 'CLE'}, {'team_id': 7, 'name': 'Mavericks', 'city': 'Dallas', 'abrv': 'DAL'}, {'team_id': 8, 'name': 'Nuggets', 'city': 'Denver', 'abrv': 'DEN'}  ... displaying 10 of 30 total bound parameter sets ...  {'team_id': 29, 'name': 'Jazz', 'city': 'Utah', 'abrv': 'UTA'}, {'team_id'

The following block is a helper function to view the results of queries in a simple format.

## Basic Queries

In [14]:
def read_query(text: sqlalchemy.sql.expression.TextClause) -> pd.DataFrame:
    '''Takes a neutral TextClause statement from SQLAlchemy
       and creates a ResultProxy object from a connection.execute()
       call. This returns a pandas DataFrame with the result of
       the executed query.'''
    with engine.connect() as connection:
        try:
            return pd.read_sql(text, connection)
        except Exception as e:
            print("There was an error. Check that your query is valid:\n{}".format(e))

### Query 2a:

Write a SQL query that can calculate team win-loss records, sorted by win percentage (defined as wins divided by games played)

In [15]:
# Creating some helper views to answer the following queries.
from sqlalchemy.exc import ProgrammingError
from sqlalchemy import text

# home_games will contain the number of home games, home wins, home losses for each team.
query_view_home_games = text("""
CREATE OR REPLACE VIEW home_games AS 
    SELECT home_id AS team_id, COUNT(*) AS total_home_games, 
           SUM(CASE WHEN home_score > away_score THEN 1 ELSE 0 END) AS home_wins, 
           SUM(CASE WHEN home_score < away_score THEN 1 ELSE 0 END) AS home_losses 
    FROM game_schedule 
    GROUP BY home_id""")

# away_games will contain the number of away games, away wins, away losses for each team
query_view_away_games = text("""
CREATE OR REPLACE VIEW away_games AS 
    SELECT away_id AS team_id, COUNT(*) AS total_away_games, 
           SUM(CASE WHEN away_score > home_score THEN 1 ELSE 0 END) away_wins, 
           SUM(CASE WHEN away_score < home_score THEN 1 ELSE 0 END) away_losses 
    FROM game_schedule 
    GROUP BY away_id""")

# total_records will contain the totals for each team
query_view_total_records = text("""
CREATE OR REPLACE VIEW total_records AS 
    SELECT home_games.team_id, home_wins + away_wins AS wins, 
                               home_losses + away_losses AS losses 
    FROM home_games JOIN away_games ON home_games.team_id = away_games.team_id""")

with engine.connect() as connection:
    try:
        connection.execute(query_view_home_games)
        connection.execute(query_view_away_games)
        connection.execute(query_view_total_records)
        print("Views created or replaced")
    except Exception as e:
        print("Something went wrong: {}".format(e))

Views created or replaced


In [16]:
query_2_a = text("""
SELECT *, CAST(CAST(wins AS DECIMAL)/(wins + losses) AS DECIMAL(6,5)) AS win_percentage
FROM team JOIN total_records ON team.team_id = total_records.team_id 
ORDER BY win_percentage DESC, team.team_id ASC""")

In [17]:
read_query(query_2_a)

,team_id,name,city,abrv,team_id,wins,losses,win_percentage
0,17,Bucks,Milwaukee,MIL,17,7,0,1.00000
1,10,Warriors,Golden State,GSW,10,8,1,0.88889
2,28,Raptors,Toronto,TOR,28,7,1,0.87500
3,8,Nuggets,Denver,DEN,8,6,1,0.85714
4,2,Celtics,Boston,BOS,2,5,2,0.71429
5,25,Trail Blazers,Portland,POR,25,5,2,0.71429
6,27,Spurs,San Antonio,SAS,27,5,2,0.71429
7,15,Grizzlies,Memphis,MEM,15,4,2,0.66667
8,12,Pacers,Indiana,IND,12,5,3,0.62500
9,26,Kings,Sacramento,SAC,26,5,3,0.62500


### Query 2b:

In the same table, show how the team ranks (highest to lowest) in terms of games played, home games,
and away games during this month of the season? Make sure your code can extend to additional months
as data is added to the data set. For each show both the number of games and the rank.

**Luckily, with the `home_games` and `away_games` views that we used in the previous questions, this should be a much simpler calculation.**


In [18]:
query_2_b = text("""
SELECT RANK () OVER ( 
       ORDER BY h.total_home_games + a.total_away_games DESC, 
                h.total_home_games DESC, 
                a.total_away_games DESC) rank, 
       h.team_id, 
       t.name, 
       h.total_home_games + a.total_away_games as total_games, 
       h.total_home_games, 
       a.total_away_games 
FROM home_games h JOIN away_games a 
        ON h.team_id = a.team_id 
    JOIN team t ON h.team_id = t.team_id 
ORDER BY total_games DESC, h.total_home_games DESC, 
    a.total_away_games DESC""")

In [19]:
read_query(query_2_b)

,rank,team_id,name,total_games,total_home_games,total_away_games
0,1,10,Warriors,9,4,5
1,2,28,Raptors,8,6,2
2,3,18,Timberwolves,8,5,3
3,3,20,Knicks,8,5,3
4,5,14,Lakers,8,4,4
5,5,5,Bulls,8,4,4
6,5,23,76ers,8,4,4
7,8,26,Kings,8,3,5
8,8,3,Nets,8,3,5
9,8,4,Hornets,8,3,5


## Schedule SQL Queries

The following queries answer the "Schedule SQL Queries" questions.

### Query 3a

The NBA has a concept of a Back-to-Back (B2B) which is if a team played 2 days in a row (regardless of start time). For the data given which team had the most Home-Home B2Bs? Which had the most Away- Away B2Bs? For example, ATL’s game on 10/30 at CLE would be an Away-Away B2B since ATL played 10/29 at PHI.

In [20]:
# Finding B2Bs for home and away ids.

# Function that finds the number of rest days between two timestamps without time zone
query_create_function_rest_days = text("""
CREATE OR REPLACE FUNCTION
    rest_days (day2 timestamp without time zone, day1 timestamp without time zone)
RETURNS integer AS $rest_days$
BEGIN
    RETURN date_part('day'::text, day2) - date_part('day'::text, day1) - 1;
END;
$rest_days$ LANGUAGE plpgsql;""")

# Home rest days
query_view_home_rest = text("""
CREATE OR REPLACE VIEW home_rest AS 
SELECT g1.home_id, g1.game_id first_game_id, g1.game_date first_game_date, 
       CAST(g2.game_id AS INTEGER) second_game_id, MIN(g2.game_date) second_game_date, 
       rest_days(min(g2.game_date), g1.game_date) 
FROM game_schedule g1 
    LEFT JOIN game_schedule g2 ON g1.home_id = g2.home_id 
                               AND g2.game_date > g1.game_date 
WHERE g2.game_id IS NOT NULL 
GROUP BY g1.home_id, first_game_id, second_game_id"""
)

# Away rest days
query_view_away_rest = text("""
CREATE OR REPLACE VIEW away_rest AS 
SELECT g1.away_id, g1.game_id first_game_id, g1.game_date first_game_date, 
       CAST(g2.game_id AS INTEGER) second_game_id, MIN(g2.game_date) second_game_date, 
       rest_days(min(g2.game_date), g1.game_date) 
FROM game_schedule g1 
    LEFT JOIN game_schedule g2 ON g1.away_id = g2.away_id 
                               AND g2.game_date > g1.game_date 
WHERE g2.game_id IS NOT NULL 
GROUP BY g1.away_id, first_game_id, second_game_id"""
)

with engine.connect() as connection:
    try:
        connection.execute(query_create_function_rest_days)
        connection.execute(query_view_home_rest)
        connection.execute(query_view_away_rest)
        print("Queries and functions created or replaced")
    except Exception as e:
        print("Something went wrong: {}".format(e))

Queries and functions created or replaced


Views now make finding the teams with back-to-backs (`rest_days = 0`) much easier. The next two blocks show the number of back to backs for both home and away games.

In [21]:
query_3_a_home = text("""
SELECT home_id, team.name, COUNT(*) num_home_b2b 
FROM home_rest JOIN team ON home_id = team.team_id 
WHERE rest_days = 0 
GROUP BY home_id, team.name"""
)
read_query(query_3_a_home)

,home_id,name,num_home_b2b
0,8,Nuggets,1
1,19,Pelicans,1


In [22]:
query_3_a_away = text("""
SELECT away_id, team.name, COUNT(*) num_away_b2b
FROM away_rest JOIN team ON away_id = team.team_id 
WHERE rest_days = 0 
GROUP BY away_id, team.name"""
)
read_query(query_3_a_away)

,away_id,name,num_away_b2b
0,1,Hawks,1
1,2,Celtics,1
2,4,Hornets,1
3,5,Bulls,1
4,9,Pistons,1
5,10,Warriors,1
6,11,Rockets,1
7,16,Heat,1
8,23,76ers,1
9,24,Suns,1


### Query 3b

Which team(s) had the longest rest between 2 games and what were the days of the 2 games?

In [23]:
query_view_games_by_team = text("""
CREATE OR REPLACE VIEW games_by_team AS (
SELECT t.team_id, g2.game_id, g2.game_date
FROM team t
    RIGHT JOIN game_schedule g2 ON t.team_id = g2.away_id
UNION ALL
SELECT t.team_id, g1.game_id, g1.game_date
FROM team t
    RIGHT JOIN game_schedule g1 ON t.team_id = g1.home_id
ORDER BY team_id ASC, game_id ASC)""")

with engine.connect() as connection:
    try:
        connection.execute(query_view_games_by_team)
        print("View created or replaced")
    except Exception as e:
        print("Something went wrong: {}".format(e))

View created or replaced


In [24]:
query_3_b = text("""
SELECT g1.team_id,
       g1.game_id AS first_game_id,
       g1.game_date AS first_game_date,
       min(g2.game_id) AS second_game_id,
       min(g2.game_date) AS second_game_date,
       rest_days(min(g2.game_date), g1.game_date) AS rest_days
FROM games_by_team g1
    LEFT JOIN games_by_team g2 ON g1.team_id = g2.team_id AND g2.game_date > g1.game_date
WHERE g2.game_id IS NOT NULL
GROUP BY g1.team_id, g1.game_id, g1.game_date
ORDER BY rest_days DESC, g1.team_id ASC, g1.game_id ASC""")

In [25]:
read_query(query_3_b)

,team_id,first_game_id,first_game_date,second_game_id,second_game_date,rest_days
0,3,26,2018-10-20 19:00:00,53,2018-10-24 19:00:00,3
1,8,66,2018-10-25 22:30:00,95,2018-10-29 21:00:00,3
2,11,70,2018-10-26 20:00:00,101,2018-10-30 20:00:00,3
3,16,31,2018-10-20 20:00:00,54,2018-10-24 19:30:00,3
4,19,21,2018-10-19 20:00:00,50,2018-10-23 20:00:00,3
...,...,...,...,...,...,...
185,26,51,2018-10-23 21:00:00,61,2018-10-24 22:00:00,0
186,26,89,2018-10-29 19:30:00,98,2018-10-30 19:00:00,0
187,28,22,2018-10-19 20:00:00,27,2018-10-20 19:00:00,0
188,28,92,2018-10-29 20:00:00,100,2018-10-30 19:30:00,0


## Lineup Queries

The following blocks discusses the lineup query questions.

### Query 4a

Notice that in the lineup data each row corresponds to a given player, game, lineup_num, period. Write a
SQL query that creates a “wide” table for the team (so a given row is now game_id, team_id, lineup_num, period, time_in, time_out, and the 5 players on the court)


In [26]:
query_view_five_player_lineup = text("""
CREATE OR REPLACE VIEW five_player_lineup AS (
SELECT game_id, team_id, lineup_num, period, array_agg(player_id) as five_players
FROM lineup
GROUP BY game_id, team_id, lineup_num, period
LIMIT 100
);""")

with engine.connect() as connection:
    try:
        connection.execute(query_view_five_player_lineup)
        print("View created or replaced")
    except Exception as e:
        print("Something went wrong: {}".format(e))

View created or replaced


In [27]:
query_4_a = text("""
SELECT f.*, l.time_in, l.time_out
FROM five_player_lineup f
    JOIN lineup l ON f.game_id = l.game_id
                  AND f.team_id = l.team_id
                  AND f.lineup_num = l.lineup_num
                  AND f.period = l.period;
""")

In [28]:
read_query(query_4_a)

,game_id,team_id,lineup_num,period,five_players,time_in,time_out
0,1,23,1,1,"[331, 288, 205, 204, 159]",720.0,411.0
1,1,23,2,1,"[331, 205, 42, 288, 159]",411.0,411.0
2,1,23,3,1,"[42, 159, 288, 146, 331]",411.0,359.0
3,1,23,4,1,"[146, 159, 42, 288, 331]",359.0,359.0
4,1,23,5,1,"[331, 146, 159, 42, 288]",359.0,267.0
...,...,...,...,...,...,...,...
495,1,2,59,4,"[228, 268, 362, 182, 327]",87.0,87.0
496,1,2,60,4,"[327, 182, 268, 228, 362]",87.0,87.0
497,1,2,61,4,"[228, 182, 327, 362, 268]",87.0,49.9
498,1,2,62,4,"[222, 228, 268, 327, 362]",49.9,49.9


### Query 4b

The field `lineup_num` changes as a player on either team gets substituted. Write a SQL query with the resultant table that stores when a player is continuously on the court for a given period (call this a stint)

In [29]:
# Helper views to reach the answer for query 4b.
query_view_all_next_times = text("""
CREATE OR REPLACE VIEW all_next_times AS (
    SELECT l1.team_id, l1.player_id, l1.game_id, l1.period, l1.lineup_num, l1.time_in, l1.time_out, (CASE WHEN l1.time_out != 0 THEN l2.time_in ELSE 0 END) next_in_or_end
    FROM lineup l1 JOIN lineup l2 ON  l1.team_id = l2.team_id
                                  AND l1.player_id = l2.player_id
                                  AND l1.game_id = l2.game_id
                                  AND l1.period = l2.period
                                  AND l1.lineup_num != l2.lineup_num
                                  AND l1.time_in != l1.time_out
    WHERE l1.time_in >= l2.time_in OR l1.time_out = 0
    GROUP BY l1.team_id, l1.player_id, l1.game_id, l1.period, l1.lineup_num, l1.time_in, l1.time_out, next_in_or_end
    ORDER BY team_id, player_id, game_id, period, l1.time_in DESC, time_out DESC, next_in_or_end DESC
);""")

query_view_next_time_in_or_end = text("""
CREATE OR REPLACE VIEW next_time_in_or_end AS (
    SELECT team_id, player_id, game_id, period, lineup_num, time_in, time_out, MAX(next_in_or_end) max_next_in_or_end
    FROM all_next_times
    WHERE time_in != next_in_or_end
    GROUP BY team_id, player_id, game_id, period, lineup_num, time_in, time_out
    ORDER BY player_id, game_id, period, lineup_num, time_in DESC, time_out DESC
);""")

query_view_created_group_stints = text("""
CREATE OR REPLACE VIEW created_group_stints AS (
SELECT team_id, player_id, game_id, period, time_in, time_out, max_next_in_or_end,
       CASE WHEN time_out = max_next_in_or_end THEN 'cont' ELSE 'end' END cont_or_end,
       COALESCE (SUM(CASE WHEN time_out != max_next_in_or_end THEN 1 ELSE 0 END) 
           OVER (PARTITION BY team_id, player_id, game_id, period 
                 ORDER BY team_id,
                          player_id, 
                          game_id, 
                          period, 
                          time_in DESC, 
                          time_out DESC, 
                          max_next_in_or_end DESC 
                 rows between unbounded preceding and 1 preceding),
                 0) AS stint_group_num
FROM next_time_in_or_end
GROUP BY team_id, player_id, game_id, period, time_in, time_out, max_next_in_or_end
ORDER BY team_id, player_id, game_id, period, time_in DESC, time_out DESC, max_next_in_or_end DESC
)
""")

query_view_stint_times = text("""
CREATE OR REPLACE VIEW stint_times AS
(
SELECT team_id, player_id,
       game_id, period, stint_group_num,
       ROW_NUMBER () OVER (PARTITION BY team_id, player_id, game_id ORDER BY team_id,
                                                                             player_id, 
                                                                             game_id, 
                                                                             period,
                                                                             stint_group_num)
            AS stint_number,
        MAX(time_in) stint_start_time,
        MIN(time_out) stint_end_time 
FROM created_group_stints
GROUP BY team_id, player_id, game_id, period, stint_group_num
)
""")

with engine.connect() as connection:
    try:
        connection.execute(query_view_all_next_times)
        connection.execute(query_view_next_time_in_or_end)
        connection.execute(query_view_created_group_stints)
        connection.execute(query_view_stint_times)
        print("Views created or replaced")
    except Exception as e:
        print("Something went wrong: {}".format(e))

Views created or replaced


In [30]:
query_4_b = text("""
SELECT g.game_date, s.team_id as team,
       CASE WHEN g.home_id != s.team_id THEN g.home_id ELSE g.away_id END as opponent,
       p.first_name || ' ' || p.last_name as player_name,
       s.period, s.stint_number,
       TO_CHAR((ROUND(s.stint_start_time) || ' seconds')::interval , 'MI:SS') AS stint_start_time,
       TO_CHAR((ROUND(s.stint_end_time) || ' seconds')::interval, 'MI:SS') AS stint_end_time
FROM stint_times s
    LEFT JOIN player p ON s.player_id = p.player_id
    LEFT JOIN game_schedule g ON s.game_id = g.game_id;
""")
read_query(query_4_b)

,game_date,team,opponent,player_name,period,stint_number,stint_start_time,stint_end_time
0,2018-10-17 19:30:00,1,20,Kent Bazemore,1,1,12:00,06:47
1,2018-10-17 19:30:00,1,20,Kent Bazemore,1,2,04:45,02:50
2,2018-10-17 19:30:00,1,20,Kent Bazemore,2,3,08:40,03:09
3,2018-10-17 19:30:00,1,20,Kent Bazemore,3,4,12:00,00:00
4,2018-10-17 19:30:00,1,20,Kent Bazemore,4,5,12:00,04:53
...,...,...,...,...,...,...,...,...
8757,2018-10-30 20:00:00,30,15,Kelly Oubre,3,4,05:18,00:00
8758,2018-10-30 20:00:00,30,15,Kelly Oubre,4,5,12:00,00:00
8759,2018-10-24 22:30:00,30,10,Thomas Bryant,4,1,09:20,00:00
8760,2018-10-28 21:30:00,30,13,Thomas Bryant,4,1,10:39,00:00


### Query 4c

From you answer to 4.b, for each player, calculate the average number of stints a player has and average stint length for a player for a given game.

In [34]:
query_4_c = text("""
SELECT team_id, player_id, game_id,
       COUNT(stint_number) AS num_stints,
       TO_CHAR((ROUND(AVG(stint_start_time - stint_end_time)) || ' seconds')::interval , 'MI:SS' ) AS avg_stint_length
FROM stint_times
GROUP BY team_id, player_id, game_id;
""")

In [35]:
read_query(query_4_c)

,team_id,player_id,game_id,num_stints,avg_stint_length
0,1,29,7,5,06:21
1,1,29,19,4,06:53
2,1,29,36,4,07:25
3,1,29,52,4,07:11
4,1,29,76,4,10:16
...,...,...,...,...,...
2351,30,227,86,3,07:26
2352,30,227,102,5,05:46
2353,30,336,62,1,09:20
2354,30,336,86,1,10:39


### Query 4d

Extend the query from 4.c to show columns for all games, in wins, in losses as well as a column that shows the difference in wins and losses

In [37]:
query_4_d = text("""
WITH query_4_c AS (
SELECT team_id, player_id, game_id,
       COUNT(stint_number) AS num_stints,
       TO_CHAR((ROUND(AVG(stint_start_time - stint_end_time)) || ' seconds')::interval , 'MI:SS' ) AS avg_stint_length
FROM stint_times
GROUP BY team_id, player_id, game_id
), game_by_winner AS (
SELECT q.*, 
       CASE WHEN q.team_id = g.home_id THEN 'home' ELSE 'away' END AS home_or_away,
       g.home_score, g.away_score
FROM query_4_c q
    LEFT JOIN game_schedule g ON q.game_id = g.game_id
)
SELECT *,
       CASE WHEN home_or_away = 'home' THEN
            CASE WHEN home_score > away_score THEN 'win' ELSE 'loss' END
       ELSE
            CASE WHEN home_score < away_score THEN 'win' ELSE 'loss' END
       END AS win_or_loss
FROM game_by_winner;
""")

In [38]:
read_query(query_4_d)

,team_id,player_id,game_id,num_stints,avg_stint_length,home_or_away,home_score,away_score,win_or_loss
0,1,29,7,5,06:21,away,126,107,loss
1,1,29,19,4,06:53,away,131,117,loss
2,1,29,36,4,07:25,away,111,133,win
3,1,29,52,4,07:11,home,111,104,win
4,1,29,76,4,10:16,home,85,97,loss
...,...,...,...,...,...,...,...,...,...
2351,30,227,86,3,07:26,away,136,104,loss
2352,30,227,102,5,05:46,away,107,95,loss
2353,30,336,62,1,09:20,away,144,122,loss
2354,30,336,86,1,10:39,away,136,104,loss


# DANGER ZONE

In [ ]:
# Remove certain SQL views
# with engine.connect() as connection:
#     connection.execute("DROP VIEW away_b2b")

In [ ]:
# Delete certain tables that need to be reinserted.
# Base.metadata.drop_all(engine, tables=[])   